In [1]:
import pandas as pd
from datasets import load_dataset
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv("HF_TOKEN_read")

from huggingface_hub import login
login(token=huggingface_token)
import openpyxl

In [2]:
dataset = load_dataset('BaekSeungJu/OphtimusEval-Dataset')
print(dataset)
# print(dataset['train'][idx]['answer'])

DatasetDict({
    train: Dataset({
        features: ['question', 'option_a', 'option_b', 'option_c', 'option_d', 'option_e', 'answer', 'answer_text'],
        num_rows: 2152
    })
})


In [ ]:
for idx in range(len(dataset['train'])):
    print(dataset['train'][idx]['question'])
    print(dataset['train'][idx]['option_a'])
    print(dataset['train'][idx]['option_b'])
    print(dataset['train'][idx]['option_c'])
    print(dataset['train'][idx]['option_d'])
    print(dataset['train'][idx]['option_e'])
    print(dataset['train'][idx]['answer_text'])
    print("--------------------------------")


In [4]:
def load_excel_workbook(excel_path):
    return openpyxl.load_workbook(excel_path)

def extract_answers_from_sheet(sheet, column_name='answer'):
    answers = []
    
    # 첫 번째 행에서 열 이름을 찾음
    header = next(sheet.iter_rows(min_row=1, max_row=1, values_only=True))
    try:
        column_index = header.index(column_name)
    except ValueError:
        raise ValueError(f"'{column_name}' is not a valid column name in the sheet.")
    
    for row in sheet.iter_rows(min_row=2, values_only=True):
        answer = row[column_index]
        if answer:
            answers.append(answer)
    
    return answers

def extract_answer_letter(input_string):
    answer_prefix = "Answer:\n"
    incorrect_prefix = "Incorrect"
    start_index = input_string.find(answer_prefix)
    if start_index == -1:
        # print(f"Warning: '{answer_prefix}' not found in the input string. Skipping this entry.")
        return incorrect_prefix  # 기본값을 반환하거나 None을 반환하여 처리

    start_index += len(answer_prefix)
    start_index = next((i for i in range(start_index, len(input_string)) if input_string[i] != ' '), start_index)

    return input_string[start_index] if start_index < len(input_string) else incorrect_prefix

def Compare_Answers(gt_answer, model_answer):
    if gt_answer.lower() == model_answer.lower():
        return True
    else:
        return False

In [5]:
exel_list = ["OphtimusEval_Ophtimus-1B-Instruct-v1-1epoch.xlsx", 
             "OphtimusEval_Ophtimus-1B-Instruct-v1-2epoch.xlsx", 
             "OphtimusEval_Ophtimus-1B-Instruct-v1-3epoch.xlsx", 
             "OphtimusEval_Ophtimus-1B-Instruct-v1-4epoch.xlsx", 
             "OphtimusEval_Ophtimus-1B-Instruct-v1-5epoch.xlsx",

             "OphtimusEval_Ophtimus-3B-Instruct-v1-1epoch.xlsx",
             "OphtimusEval_Ophtimus-3B-Instruct-v1-2epoch.xlsx",
             "OphtimusEval_Ophtimus-3B-Instruct-v1-3epoch.xlsx",
             "OphtimusEval_Ophtimus-3B-Instruct-v1-4epoch.xlsx",
             "OphtimusEval_Ophtimus-3B-Instruct-v1-5epoch.xlsx",

             "OphtimusEval_Ophtimus-8B-Instruct-v1-1epoch.xlsx",
             "OphtimusEval_Ophtimus-8B-Instruct-v1-2epoch.xlsx",
             "OphtimusEval_Ophtimus-8B-Instruct-v1-3epoch.xlsx",
             "OphtimusEval_Ophtimus-8B-Instruct-v1-4epoch.xlsx",
             "OphtimusEval_Ophtimus-8B-Instruct-v1-5epoch.xlsx",
             ]

In [ ]:
for exel in exel_list:
    correct_count = 0
    incorrect_count = 0
    incorrect_indices = []  # incorrect 인덱스를 저장할 리스트
    workbook = load_excel_workbook(f'./Dataset/{exel}')
    sheet = workbook['Sheet1']
    answers = extract_answers_from_sheet(sheet, column_name='answer')
    for idx in range(len(dataset['train'])):
        gt_answer = dataset['train'][idx]['answer']
        model_answer = extract_answer_letter(answers[idx])
        if Compare_Answers(gt_answer, model_answer):
            correct_count += 1
        else:
            incorrect_count += 1
            # incorrect_indices.append(idx+1)
            print(f"Incorrect Index: {idx+1}")
            print(f"GT Answer: {gt_answer}, Model Answer: {model_answer}")  # incorrect 인덱스를 리스트에 추가
    total_count = correct_count + incorrect_count
    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    print(exel[:-5])
    print(f"Correct: {correct_count}, Incorrect: {incorrect_count}, Accuracy: {accuracy:.2f}%")
    # print(f"Incorrect Indices: {incorrect_indices}")  # incorrect 인덱스 출력
    print("--------------------------------")